<a href="https://colab.research.google.com/github/arunpothencode/MyAIApplication/blob/dev20Nov24/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [2]:
!pip install -U -q "google-generativeai>=0.8.2"

In [3]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkhlcmUncyBhIFB5dGhvbiBmdW5jdGlvbiB0aGF0IHNvcnRzIGEgbGlzdCBvZiBudW1iZXJzIGluIGFzY2VuZGluZyBvcmRlci4gSWRlbnRpZnkgdGhlIHRpbWUgY29tcGxleGl0eSBvZiB0aGlzIGZ1bmN0aW9uLiBUaGVuIG9wdGltaXplIHRoaXMgZnVuY3Rpb24gYW5kIGV4cGxhaW4gd2h5IGl0IGlzIGZhc3Rlci4gRG9uJ3QgdXNlIFB5dGhvbidzIGluLWJ1aWx0IHNvcnQoKSBtZXRob2QuXFxuXFxuXFxuYGBgZGVmIHNvcnRfbGlzdChsaXN0X3RvX3NvcnQpOlxcbiAgXFxcIlxcXCJcXFwiXFxuICBUaGlzIGZ1bmN0aW9uIHNvcnRzIGEgbGlzdCBvZiBudW1iZXJzIGluIGFzY2VuZGluZyBvcmRlciB1c2luZyB0aGUgYnViYmxlIHNvcnQgYWxnb3JpdGhtLlxcblxcbiAgQXJnczpcXG4gICAgbGlzdF90b19zb3J0OiBBIGxpc3Qgb2YgbnVtYmVycyB0byBiZSBzb3J0ZWQuXFxuXFxuICBSZXR1cm5zOlxcbiAgICBBIG5ldyBsaXN0IHdpdGggdGhlIG51bWJlcnMgc29ydGVkIGluIGFzY2VuZGluZyBvcmRlci5cXG4gIFxcXCJcXFwiXFxcIlxcbiAgIyBDcmVhdGUgYSBjb3B5IG9mIHRoZSBsaXN0IHRvIGF2b2lkIG1vZGlmeWluZyB0aGUgb3JpZ2luYWxcXG4gIHNvcnRlZF9saXN0ID0gbGlzdF90b19zb3J0LmNvcHkoKVxcbiAgbiA9IGxlbihzb3J0ZWRfbGlzdClcXG5cXG4gICMgSXRlcmF0ZSB0aHJvdWdoIHRoZSBsaXN0IG4tMSB0aW1lc1xcbiAgZm9yIGkgaW4gcmFuZ2Uobi0xKTpcXG4gICAgIyBGbGFnIHRvIHRyYWNrIGlmIGFueSBzd2FwcyB3ZXJlIG1hZGUgaW4gYSBwYXNzXFxuICAgIHN3YXBwZWQgPSBGYWxzZVxcbiAgICAjIEl0ZXJhdGUgdGhyb3VnaCB0aGUgdW5zb3J0ZWQgcG9ydGlvbiBvZiB0aGUgbGlzdFxcbiAgICBmb3IgaiBpbiByYW5nZShuLWktMSk6XFxuICAgICAgIyBDb21wYXJlIGFkamFjZW50IGVsZW1lbnRzIGFuZCBzd2FwIGlmIG5lY2Vzc2FyeVxcbiAgICAgIGlmIHNvcnRlZF9saXN0W2pdID4gc29ydGVkX2xpc3RbaisxXTpcXG4gICAgICAgIHNvcnRlZF9saXN0W2pdLCBzb3J0ZWRfbGlzdFtqKzFdID0gc29ydGVkX2xpc3RbaisxXSwgc29ydGVkX2xpc3Rbal1cXG4gICAgICAgIHN3YXBwZWQgPSBUcnVlXFxuICAgICMgSWYgbm8gc3dhcHMgd2VyZSBtYWRlLCB0aGUgbGlzdCBpcyBhbHJlYWR5IHNvcnRlZFxcbiAgICBpZiBub3Qgc3dhcHBlZDpcXG4gICAgICBicmVha1xcblxcbiAgIyBSZXR1cm4gdGhlIHNvcnRlZCBsaXN0XFxuICByZXR1cm4gc29ydGVkX2xpc3RcXG5cXG4jIEV4YW1wbGUgdXNhZ2VcXG5teV9saXN0ID0gWzEsIDksIDUsIDIsIDEsIDgsIDYsIDYsIDMsIDQsIDEwLCA3XVxcbnNvcnRlZF9saXN0ID0gc29ydF9saXN0KG15X2xpc3QpXFxucHJpbnQoc29ydGVkX2xpc3QpICAjIE91dHB1dDogWzEsIDEsIDIsIDMsIDQsIDUsIDYsIDYsIDcsIDgsIDksIDEwXVxcbmBgYCJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "Here's a Python function that sorts a list of numbers in ascending order. Identify the time complexity of this function. Then optimize this function and explain why it is faster. Don't use Python's in-built sort() method.\\n\\n\\n```def sort_list(list_to_sort):\\n  \\\"\\\"\\\"\\n  This function sorts a list of numbers in ascending order using the bubble sort algorithm.\\n\\n  Args:\\n    list_to_sort: A list of numbers to be sorted.\\n\\n  Returns:\\n    A new list with the numbers sorted in ascending order.\\n  \\\"\\\"\\\"\\n  # Create a copy of the list to avoid modifying the original\\n  sorted_list = list_to_sort.copy()\\n  n = len(sorted_list)\\n\\n  # Iterate through the list n-1 times\\n  for i in range(n-1):\\n    # Flag to track if any swaps were made in a pass\\n    swapped = False\\n    # Iterate through the unsorted portion of the list\\n    for j in range(n-i-1):\\n      # Compare a

## Call `generate_content`

In [4]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

The provided Python function uses the Bubble Sort algorithm.  The time complexity of Bubble Sort is O(n^2) in the worst and average cases, and O(n) in the best case (when the list is already sorted).  This is because it involves nested loops, each iterating through the list (or a portion of it).

Here's an optimized version using the Merge Sort algorithm, which has a time complexity of O(n log n) in all cases:

```python
def merge_sort(list_to_sort):
    """
    This function sorts a list of numbers in ascending order using the merge sort algorithm.

    Args:
      list_to_sort: A list of numbers to be sorted.

    Returns:
      A new list with the numbers sorted in ascending order.
    """
    if len(list_to_sort) <= 1:
        return list_to_sort

    mid = len(list_to_sort) // 2
    left_half = list_to_sort[:mid]
    right_half = list_to_sort[mid:]

    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    return merge(left_half, right_half)

def merge(left, right):
    merged = []
    i = j = 0

    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            merged.append(left[i])
            i += 1
        else:
            merged.append(right[j])
            j += 1

    merged.extend(left[i:])
    merged.extend(right[j:])
    return merged

# Example usage
my_list = [1, 9, 5, 2, 1, 8, 6, 6, 3, 4, 10, 7]
sorted_list = merge_sort(my_list)
print(sorted_list)  # Output: [1, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 10]
```

**Why Merge Sort is faster:**

Merge Sort is significantly faster than Bubble Sort for larger lists because of its time complexity.  While Bubble Sort makes many comparisons and swaps of adjacent elements, potentially repeatedly comparing and swapping the same elements, Merge Sort employs a "divide and conquer" strategy:

1. **Divide:** It recursively divides the list into smaller sublists until each sublist contains only one element (which is inherently sorted).

2. **Conquer:** It then repeatedly merges the sublists to produce new sorted sublists until there is only one sorted list remaining.  The merging process is efficient because it only needs to compare elements from the two sublists once.

The O(n log n) complexity of Merge Sort means that the number of operations grows proportionally to n multiplied by the logarithm of n.  This is considerably slower than the n² growth of Bubble Sort.  For large values of 'n', the difference in performance is dramatic.  While Merge Sort has a larger constant overhead (due to recursion), this is dwarfed by the reduction in the number of comparisons and swaps for larger input sizes.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [5]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Here's a Python function that sorts a list of numbers in ascending order. Identify the time complexity of this function. Then optimize this function and explain why it is faster. Don't use Python's in-built sort() method.\n\n\n```def sort_list(list_to_sort):\n  \"\"\"\n  This function sorts a list of numbers in ascending order using the bubble sort algorithm.\n\n  Args:\n    list_to_sort: A list of numbers to be sorted.\n\n  Returns:\n    A new list with the numbers sorted in ascending order.\n  \"\"\"\n  # Create a copy of the list to avoid modifying the original\n  sorted_list = list_to_sort.copy()\n  n = len(sorted_list)\n\n  # Iterate through the list n-1 times\n  for i in range(n-1):\n    # Flag to track if any swaps were made in a pass\n    swapped = False\n    # Iterate through the unsorted portion of the list\n    for j in range(n-i-1):\n      # Compare adjacent elements and swap if necessary\n      if sorted_list[j] > sorted_list[j+1]:\n        sorted_list[j], sor